# Fully connected Auto-Encoder for Tabular Data
In order to compare the classical and quantum methods we will implement several methods including the deep learning method, fully connected auto-encoder. 

In [16]:
# TODO: Make this an automated file, not copying code from "SVM on Iris and arrhythmia Datasets.ipynb"
# Data processing
import pandas as pd
import numpy as np
from collections import Counter
# Visualization
import matplotlib.pyplot as plt

# Data Sets
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
from sklearn.datasets import load_breast_cancer

#Auto-encoder imports
import torchvision
from torchvision import datasets
from torchvision import transforms
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

#DataSets that we can test
#climate
liver = 'https://archive.ics.uci.edu/ml/machine-learning-databases/liver-disorders/bupa.data'
arrhythmia = 'https://archive.ics.uci.edu/ml/machine-learning-databases/arrhythmia/arrhythmia.data'
vowel = 'https://archive.ics.uci.edu/ml/machine-learning-databases/undocumented/connectionist-bench/vowel/vowel-context.data'
iris = load_iris()
breast_cancer = load_breast_cancer()

name_datasets_csv = {'arrhythmia': arrhythmia}
name_datasets_sklearn = {'iris_data': iris, 'breast_cancer_data': breast_cancer}

In [17]:
data = {}
target = {}
dimensionality = {}
datasets = {}


In [23]:
vowel_data = pd.read_csv(vowel,skiprows=2, index_col=False,delim_whitespace=True)
vowel_data.columns = ['feature{}'.format(i) for i in range(0,vowel_data.shape[1])]

In [27]:
#which class is anomaly?
vowel_data[vowel_data.columns[-1:]]

,feature13
0,3
1,4
2,5
3,6
4,7
...,...
982,6
983,7
984,8
985,9


In [41]:
from torch.utils.data import Dataset
train_data, val_data, test_data = torch.utils.data.random_split(vowel_data, [int(len(vowel_data)*0.6), int(len(vowel_data)*0.2)+1, int(len(vowel_data)*0.2)],generator=torch.Generator().manual_seed(42))
# dimensionality['vowel_data'] = [vowel_data.shape,vowel_data.nunique().tolist()[-1:]]
# name_list = ['train_data', 'val_data', 'test_data']

# for i, data_split in enumerate([train_data, val_data, test_data]):
#     print(data_split)
#     data[name_list[i]] = data_split[data_split.columns[:-1]]
#     target[name_list[i]] = data_split[data_split.columns[-1:]]
class NumbersDataset(Dataset):
    def __init__(self):
        self.samples = list(range(1, 1001))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx]
    
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=64)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64)
for data in train_loader:
    print(data)
    break
#FIXME: Does it make sense to include both 0 and 1 target values at training?

KeyError: 267

In [39]:
# Define the autoencoder class
class AutoEncoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(AutoEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size//2),
            nn.ReLU(),
            nn.Linear(hidden_size//2, hidden_size//4)
        )
        self.decoder = nn.Sequential(
            nn.Linear(hidden_size//4, hidden_size//2),
            nn.ReLU(),
            nn.Linear(hidden_size//2, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, input_size)
        )
    
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Set up the autoencoder model
input_size = 13  # The size of the input data
hidden_size = 6  # The number of neurons in the hidden layer
autoencoder = AutoEncoder(input_size, hidden_size)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=0.01)

# Get data
# X_train = train_loader
# X_val = val_loader

# Train the autoencoder
num_epochs = 50
batch_size = 10
for epoch in range(num_epochs):
    for unprocessed_batch in train_loader:
        batch = unprocessed_batch[:, :-1]
        # Forward pass
        output = autoencoder(batch)
        loss = criterion(output, batch)
        
        # Backward pass and optimization step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
        
    # Print the loss for this epoch
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")


KeyError: 267